# Import Libraries

- import libraries นำเข้าไฟล์โปรแกรมส่วนเสริมที่เกี่ยวข้อง อาทิเช่น การแสดงผลหน้าจอ-ปุ่มกด, ดึงเวลาปัจจุบัน, การอ่าน-เขียน ไฟล์

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

# Display Live Camera Feed

- ตั้งค่ากล้องถ่ายภาพของตัวรถ และตั้งค่าการแสดงผล open cv เพื่อบันทึกจุดที่สนใจในภาพ

In [ ]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

# Collect data

- สร้าง Folder เพื่อจัดเก็บภาพถ่าย

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

- ประกาศตัวแปรเพื่ออ่านจำนวนไฟล์ภาพถ่ายปัจจุบัน

In [ ]:
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

- ประกาศฟังก์ชัน ตั้งชื่อไฟล์ภาพถ่าย โดยกำหนด x, y ลงไปในไฟล์ภาพถ่าย

In [ ]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

- สร้างส่วนแสดงผลภาพจากกล้อง และสร้างฟังก์ชันตรวจจับการกดปุ่มถ่ายภาพ

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='Take photo', button_style='success', layout=button_layout)

def save_free():
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

free_button.on_click(lambda x: save_free())

- แสดงภาพจากกล้องแบบเรียลไทม์ และบันทึกจุดที่สนใจ เพื่อให้รถทำการเรียนรู้

In [ ]:
display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

- แสดงปุ่มกดถ่ายภาพ และแสดงจำนวนภาพถ่ายปัจจุบัน

In [ ]:
display(widgets.HBox([free_button, count_widget]))

# Backup Dataset

- สำรองข้อมูลไฟล์ภาพถ่าย

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}